In [37]:
import os

In [38]:
os.chdir("/home/tanush-reddy/workspace/Wine-Grade-Estimator")

In [39]:
from dataclasses import dataclass
from pathlib import Path

In [40]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/atanushreddy/Wine-Grade-Estimator.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="atanushreddy"
os.environ["MLFLOW_TRACKING_PASSWORD"]="702b03e47aedac5981cd5ae775223366bc2b1711"


In [41]:
from dataclasses import dataclass
from pathlib import Path


In [42]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    """
    Configuration for model evaluation.
    """
    root_dir: Path 
    test_data_file_path: Path
    model_path :Path
    all_params : dict
    evaluation_report : Path
    target_column: str
    mlflow_uri: str

In [43]:
from src.constants import *
from src.utils.common import read_yaml, create_directories


In [44]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        self.config_file_path = config_file_path
        self.params_file_path = params_file_path
        self.schema_file_path = schema_file_path

        self.config = read_yaml(self.config_file_path)
        self.params = read_yaml(self.params_file_path)
        self.schema = read_yaml(self.schema_file_path)

        create_directories([self.config.artifacts_root])
        
        
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        """
        Returns the configuration for model evaluation.
        """
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_file_path=Path(config.test_data_file_path),
            model_path=Path(config.model_path),
            all_params=dict(params),
            evaluation_report=Path(config.evaluation_report),
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/atanushreddy/Wine-Grade-Estimator.mlflow"
        )

        return model_evaluation_config
    

In [45]:
import os
from src import logger
from src.exception import CustomException
import sys
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import mlflow
from urllib.parse import urlparse
import mlflow.sklearn
import numpy as np  
import joblib
from src.utils.common import save_json
import tempfile

In [46]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
        
    def eval_metrics(self, actual, predicted):
        """
        Calculate evaluation metrics.
        """
        r2 = r2_score(actual, predicted)
        mae = mean_absolute_error(actual, predicted)
        mse = mean_squared_error(actual, predicted)
        rmse = np.sqrt(mse)

        return rmse, mae, mse, r2

    def log_into_mlflow(self):
        """ Log evaluation metrics into MLflow.
        """
        try:
            test_df = pd.read_csv(self.config.test_data_file_path)
            model=joblib.load(self.config.model_path)
        
            X_test = test_df.drop(columns=[self.config.target_column],axis=1)
            y_test = test_df[self.config.target_column]
        
            mlflow.set_tracking_uri(self.config.mlflow_uri)
            mlflow.set_experiment("WineGradeEvaluation")
            tracking_uri = urlparse(mlflow.get_tracking_uri()).scheme
            
            with mlflow.start_run():
                predicted_qualities = model.predict(X_test)
                rmse, mae, mse, r2 = self.eval_metrics(y_test, predicted_qualities)
                
                scores = {
                    "rmse": rmse,
                    "mae": mae,
                    "mse": mse,
                    "r2": r2
                }
                
                save_json(file_path=Path(self.config.evaluation_report), data=scores)
                
                mlflow.log_params(self.config.all_params)
                mlflow.log_metric("rmse", rmse)
                mlflow.log_metric("mae", mae)
                mlflow.log_metric("mse", mse)
                mlflow.log_metric("r2", r2)
                
                with tempfile.TemporaryDirectory() as temp_dir:
                    model_output_path = os.path.join(temp_dir, "model")
                    os.makedirs(model_output_path, exist_ok=True)
                    joblib.dump(model, os.path.join(model_output_path, "model.pkl"))
                    mlflow.log_artifacts(model_output_path, artifact_path="model")
                
        except Exception as e:
            raise CustomException(e, sys)

In [47]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval_config = ModelEvaluation(config=model_eval_config)
    model_eval_config.log_into_mlflow()
except Exception as e:
    logger.error(f"Error during model evaluation: {e}")
    raise CustomException(e, sys)

[2025-07-30 13:03:40,461] 28 WineGradeEstimator - INFO - YAML file config/config.yaml read successfully.
[2025-07-30 13:03:40,463] 28 WineGradeEstimator - INFO - YAML file params.yaml read successfully.
[2025-07-30 13:03:40,466] 28 WineGradeEstimator - INFO - YAML file schema.yaml read successfully.
[2025-07-30 13:03:40,468] 46 WineGradeEstimator - INFO - Directory created: artifacts
[2025-07-30 13:03:40,469] 46 WineGradeEstimator - INFO - Directory created: artifacts/model_evaluation


2025/07/30 13:03:41 INFO mlflow.tracking.fluent: Experiment with name 'WineGradeEvaluation' does not exist. Creating a new experiment.


[2025-07-30 13:03:42,291] 62 WineGradeEstimator - INFO - Data saved to JSON file: artifacts/model_evaluation/evaluation_report.json
🏃 View run bright-roo-283 at: https://dagshub.com/atanushreddy/Wine-Grade-Estimator.mlflow/#/experiments/1/runs/2e133d5725774c2fb83381eeaf17b2be
🧪 View experiment at: https://dagshub.com/atanushreddy/Wine-Grade-Estimator.mlflow/#/experiments/1
